In [69]:
from tqdm import tqdm
import pandas as pd
import string
from nltk.tokenize import word_tokenize
import re

In [62]:
dataset = pd.read_pickle('unprocessed_dataset.pkl')
type(dataset)


list

In [63]:
dataset[1]['story'].replace('-',' ')

'(CNN)George Zimmerman    acquitted by a Florida jury over the death of Trayvon Martin    was arrested in Florida on suspicion of aggravated assault and domestic violence with a weapon, local authorities said.\n\nThe 31 year old Florida resident was arrested Friday by police in Lake Mary at about 10 p.m. and booked into the John E. Polk Correctional Facility, according to that facility\'s website. That facility, like its website, is run by the Seminole County Sheriff\'s Office.\n\nIt all came about after Zimmerman allegedly threw a wine bottle at a girlfriend, his lawyer Don West told reporters.\n\n"Whatever happened took place several days ago," said West. "And, as far as I know, they have not been together for some time, certainly not since then."\n\nPolice first learned about it after coming "in contact with the (alleged) victim at a traffic stop" on Monday, Lake Mary police spokeswoman Bianca Gillett  said.\n\nWhen asked about the delay in making the arrest, Gillett said the arrest

In [65]:
data = dataset[1]['story']
# custom_punctuation = "!\"#$%&()*+,-./:;<=>?@[\]^_`{|}~"
# table = str.maketrans('', '', custom_punctuation)
def clean_data(data):
    data = data.split('. ')
    table = str.maketrans('', '', string.punctuation)

    cleaned = list()
    for sentence in data:
        if len(sentence)>0:
            
            dash_indx = sentence.find('--')
            if dash_indx>=0:
                sentence = sentence[dash_indx+len('--'):]
            sentence = sentence.strip()
            sentence = sentence.replace('-',' ')
            word = word_tokenize(sentence)
            #word = [w.lower() for w in word]
            word = [w.translate(table).lower() for w in word if len(w)>1]
            cleaned.append(' '.join(word))

    data = '. '.join(cleaned)
    
    return re.sub(' +', ' ', data)
print(type(data))
clean_data(data)

<class 'str'>


'acquitted by florida jury over the death of trayvon martin was arrested in florida on suspicion of aggravated assault and domestic violence with weapon local authorities said the 31 year old florida resident was arrested friday by police in lake mary at about 10 pm. and booked into the john. polk correctional facility according to that facility s website. that facility like its website is run by the seminole county sheriff s office it all came about after zimmerman allegedly threw wine bottle at girlfriend his lawyer don west told reporters whatever happened took place several days ago said west. and as far as know they have not been together for some time certainly not since then police first learned about it after coming in contact with the alleged victim at traffic stop on monday lake mary police spokeswoman bianca gillett said when asked about the delay in making the arrest gillett said the arresting officer tried to reach zimmerman on monday and tuesday but could nt and was off d

In [70]:
for i in tqdm(range(len(dataset))):
    dataset[i]['story'] = clean_data(dataset[i]['story'])
    dataset[i]['highlight'] = clean_data(dataset[i]['highlight'])

100%|██████████| 312085/312085 [23:26<00:00, 221.91it/s]


In [71]:
df = pd.DataFrame(dataset)

In [72]:
df.isnull().sum()

story        0
highlight    0
dtype: int64

In [73]:
import pickle
output_file = open('cleaned_df.pkl','wb')
pickle.dump(df, output_file)
output_file.close()

Let us check the word_count for the articles in our dataset to understand the length of these articles

In [74]:
#dataset_pd.drop('hightlight1', axis=1, inplace=True)
df['word_count_text'] = df['story'].apply(lambda x: len(str(x).split()))

lets check for 0-100 percentile range in steps of 10

In [75]:
import numpy as np
for i in range(0,100,10) :
    var =df["word_count_text"]. values
    var = np.sort(var,axis = None)
    print (" {} percentile value is {}".format (i,var[int(len(var)*(float(i)/100))]))
print ("100 percentile value is ",var[-1])

 0 percentile value is 0
 10 percentile value is 294
 20 percentile value is 381
 30 percentile value is 455
 40 percentile value is 525
 50 percentile value is 597
 60 percentile value is 677
 70 percentile value is 772
 80 percentile value is 897
 90 percentile value is 1101
100 percentile value is  2167


checking the values in between 90 and 100. 


In [77]:
for i in range (90,100) :
    var =df["word_count_text"].values
    var = np.sort(var, axis = None)
    print("{} percentile value is {}".format(i, var[int (len(var)*(float (i)/100))]))
print ("100 percentile value is ", var[-1])

90 percentile value is 1101
91 percentile value is 1132
92 percentile value is 1166
93 percentile value is 1203
94 percentile value is 1245
95 percentile value is 1292
96 percentile value is 1348
97 percentile value is 1413
98 percentile value is 1496
99 percentile value is 1600
100 percentile value is  2167


We can assume the lenght of articles in the range of 1200-1300 as 95% articles are less than that.